## ✅ Imports & Environment


In [ ]:
import asyncio
import os
from textwrap import dedent
from typing import Dict, List, Tuple

import gradio as gr
from dotenv import load_dotenv

from agents import Agent, Runner, function_tool, WebSearchTool, trace
from agents.model_settings import ModelSettings
from openai.types.responses import ResponseTextDeltaEvent
from pydantic import BaseModel, Field
from email.mime.text import MIMEText
import smtplib

load_dotenv(override=True)

GMAIL_USER = os.getenv("GMAIL_USER")
GMAIL_APP_PASSWORD = os.getenv("GMAIL_APP_PASSWORD")
GMAIL_TO = os.getenv("GMAIL_TO")
DRY_RUN = os.getenv("DRY_RUN", "true").lower() != "false"


## ✅ Helper Utilities


In [ ]:
def run_async(coro):
    """Synchronously execute an async coroutine (for notebook buttons)."""
    return asyncio.run(coro)


def parse_numbered_list(text: str) -> List[str]:
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    questions = []
    for line in lines:
        if line[0].isdigit() and "." in line:
            _, remainder = line.split(".", 1)
            questions.append(remainder.strip())
    return questions or lines


## ✅ Agent Definitions


In [ ]:
clarifier_agent = Agent(
    name="Clarifying Question Generator",
    instructions=(
        "You create exactly three clarifying questions that will help tune research and personalization before "
        "generating a cold email. Keep them concise."
    ),
    model="gpt-4o-mini",
)


class SearchQueryPlan(BaseModel):
    query: str = Field(description="A focused web search string to run in order to personalize the outreach.")
    reason: str = Field(description="Why this search will help tailor the message.")


class LeadResearchPlan(BaseModel):
    insight_summary: str = Field(description="Three-sentence recap of the most relevant personalization angle.")
    search_queries: List[SearchQueryPlan] = Field(
        description="Exactly three web searches to perform before writing outreach.",
        min_items=3,
        max_items=3,
    )


lead_researcher = Agent(
    name="Lead Research Strategist",
    instructions=(
        "Use the clarifications to create a structured research plan. Return exactly three search_queries along "
        "with an insight_summary that ties everything together."
    ),
    model="gpt-4o-mini",
    output_type=LeadResearchPlan,
)

search_executor = Agent(
    name="Web Search Executor",
    instructions=(
        "Run the provided search query using the WebSearchTool. The input will be in the format 'Query: ...' "
        "followed by 'Reason: ...'. Incorporate the reason to focus your summary. Return bullet points (<=120 words)."
    ),
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

persona_instructions = {
    "Professional": "You write polished, data-backed cold emails for ComplAI, reinforcing audit readiness.",
    "Engaging": "You write upbeat, story-driven cold emails for ComplAI, using light humor while staying credible.",
    "Concise": "You write crisp cold emails for ComplAI, highlighting ROI in under 120 words.",
}

sales_agents = {
    persona: Agent(
        name=f"{persona} SDR",
        instructions=dedent(
            f"""
            You are a {persona.lower()} sales development rep for ComplAI.
            {tone}
            Use the clarifications and research digest provided. Output only the email body.
            """
        ).strip(),
        model="gpt-4o-mini",
    )
    for persona, tone in persona_instructions.items()
}

reviewer_agent = Agent(
    name="Email Reviewer",
    instructions=(
        "You compare multiple cold email drafts and return the single best one. Consider personalization, clarity, "
        "and the strength of the call to action. Return only the winning email body."
    ),
    model="gpt-4o-mini",
)

subject_writer = Agent(
    name="Subject Line Writer",
    instructions="Write a compelling subject line for the provided cold email body.",
    model="gpt-4o-mini",
)

html_converter = Agent(
    name="HTML Formatter",
    instructions="Convert the cold email body to a clean HTML layout with short paragraphs and clear calls to action.",
    model="gpt-4o-mini",
)


class EmailSendReport(BaseModel):
    subject: str = Field(description="Subject line used for the email.")
    plain_text_body: str = Field(description="Plain text version of the winning email body.")
    html_body: str = Field(description="HTML formatted email body.")
    send_status: str = Field(description="Status returned from send_email (sent or dry_run).")
    detail: str | None = Field(default=None, description="Additional status details.")


@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send an HTML email via Gmail SMTP. Honours DRY_RUN to avoid accidental sends."""
    if DRY_RUN:
        return {"status": "dry_run", "detail": "DRY_RUN enabled; email not sent."}

    if not all([GMAIL_USER, GMAIL_APP_PASSWORD, GMAIL_TO]):
        raise RuntimeError("GMAIL_USER, GMAIL_APP_PASSWORD, and GMAIL_TO must be set or DRY_RUN must remain true.")

    msg = MIMEText(html_body, "html")
    msg["Subject"] = subject
    msg["From"] = GMAIL_USER
    msg["To"] = GMAIL_TO

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(GMAIL_USER, GMAIL_APP_PASSWORD)
        server.sendmail(GMAIL_USER, [GMAIL_TO], msg.as_string())

    return {"status": "sent", "detail": f"Email sent to {GMAIL_TO}"}


def _email_manager_agent():
    return Agent(
        name="Email Manager",
        instructions=(
            "Receive the winning email body, generate a subject via the subject_writer tool, convert to HTML via "
            "the html_converter tool, then call send_email with both. Respond strictly using the EmailSendReport schema."
        ),
        tools=[
            subject_writer.as_tool("subject_writer", "Subject line generator"),
            html_converter.as_tool("html_converter", "Email HTML formatter"),
            send_email,
        ],
        model="gpt-4o-mini",
        output_type=EmailSendReport,
    )


## ✅ Core Async Workflow


In [ ]:
async def generate_clarifying_questions(initial_request: str) -> List[str]:
    result = await Runner.run(clarifier_agent, initial_request)
    return parse_numbered_list(result.final_output)


async def build_research_plan(initial_request: str, clarifications: Dict[str, str]) -> LeadResearchPlan:
    clarifications_text = "\n".join(f"{k}: {v}" for k, v in clarifications.items())
    message = dedent(
        f"""
        Initial request: {initial_request}
        Clarifications:
        {clarifications_text}
        """
    ).strip()
    result = await Runner.run(lead_researcher, message)
    return result.final_output


async def execute_searches(plan: LeadResearchPlan) -> List[Dict[str, str]]:
    summaries = []
    for item in plan.search_queries:
        prompt = f"Query: {item.query}\nReason: {item.reason}"
        result = await Runner.run(search_executor, prompt)
        summaries.append({
            "query": item.query,
            "reason": item.reason,
            "findings": result.final_output,
        })
    return summaries


async def draft_email_variations(plan: LeadResearchPlan, search_summaries: List[Dict[str, str]], clarifications: Dict[str, str]) -> Dict[str, str]:
    digest = dedent(
        "\n".join(
            [plan.insight_summary, "\nResearch findings:"] +
            [f"- {summary['findings']}" for summary in search_summaries]
        )
    ).strip()

    clar_text = "\n".join(f"{k}: {v}" for k, v in clarifications.items())
    message = dedent(
        f"""
        Clarifications:
        {clar_text}

        Research digest:
        {digest}
        """
    ).strip()

    drafts = {}
    for persona, agent in sales_agents.items():
        result = await Runner.run(agent, message)
        drafts[persona] = result.final_output
    return drafts


async def select_best_email(drafts: Dict[str, str]) -> str:
    combined = "\n\n".join(f"{persona} draft:\n{body}" for persona, body in drafts.items())
    result = await Runner.run(reviewer_agent, combined)
    return result.final_output


async def format_and_send(winning_body: str, actually_send: bool) -> EmailSendReport:
    global DRY_RUN
    previous_state = DRY_RUN
    DRY_RUN = not actually_send
    try:
        email_agent = _email_manager_agent()
        result = await Runner.run(email_agent, winning_body)
        return result.final_output
    finally:
        DRY_RUN = previous_state


async def run_pipeline(initial_request: str, clarifications: Dict[str, str], send_email_flag: bool):
    with trace("Research Outreach Pipeline"):
        plan = await build_research_plan(initial_request, clarifications)
        searches = await execute_searches(plan)
        drafts = await draft_email_variations(plan, searches, clarifications)
        winning = await select_best_email(drafts)
        email_result = await format_and_send(winning, send_email_flag)

    detail = email_result.detail or ""
    detail_note = f" ({detail})" if detail else ""
    markdown_email = dedent(
        f"""
        ## Subject
        {email_result.subject}

        ## Email Body
        {email_result.plain_text_body}

        _Status: {email_result.send_status}{detail_note}_
        """
    ).strip()

    return {
        "markdown_email": markdown_email,
    }


## ✅ Gradio Callbacks


In [ ]:
def on_generate_questions(initial_request: str):
    if not initial_request.strip():
        raise gr.Error("Please enter a topic or brief first.")
    questions = run_async(generate_clarifying_questions(initial_request))
    questions = questions[:3] + ["" for _ in range(max(0, 3 - len(questions)))]
    return questions[:3]


def on_run_pipeline(initial_request: str, answers: List[str], questions: List[str], send_email_flag: bool):
    if not initial_request.strip():
        raise gr.Error("Please provide the initial topic or brief.")
    if len(questions) < 3:
        raise gr.Error("Generate clarifying questions first.")

    clarifications = {
        questions[i]: answers[i] or "Not provided"
        for i in range(3)
    }

    yield "⏳ Running research and drafting email..."

    try:
        results = run_async(run_pipeline(initial_request, clarifications, send_email_flag))
    except Exception as exc:  # noqa: BLE001
        raise gr.Error(f"Pipeline failed: {exc}") from exc

    yield results["markdown_email"]


## ✅ Launch Gradio Interface


In [ ]:
with gr.Blocks(title="Research-Guided Outreach") as demo:
    gr.Markdown(
        """
        # Research-Guided Outreach Agent
        Provide a topic or company initiative and let the agent clarify, research, draft, and (optionally) send an email.
        \n> **Note:** Web searches incur OpenAI's hosted search fee. Leave **Send email** unchecked to stay in dry-run mode.
        """
    )

    topic_input = gr.Textbox(label="Topic or outreach brief", placeholder="e.g. Introduce ComplAI to Redwood Robotics")
    generate_btn = gr.Button("Generate clarifying questions")
    questions_state = gr.State([])

    answer1 = gr.Textbox(label="Question 1", placeholder="Answer to question 1")
    answer2 = gr.Textbox(label="Question 2", placeholder="Answer to question 2")
    answer3 = gr.Textbox(label="Question 3", placeholder="Answer to question 3")

    def _handle_generate(topic, current_questions):
        questions = on_generate_questions(topic)
        return (
            gr.update(label=questions[0] or "Question 1"),
            gr.update(label=questions[1] or "Question 2"),
            gr.update(label=questions[2] or "Question 3"),
            questions,
        )

    generate_btn.click(
        fn=_handle_generate,
        inputs=[topic_input, questions_state],
        outputs=[answer1, answer2, answer3, questions_state],
    )

    send_toggle = gr.Checkbox(label="Send email via Gmail", value=False)
    run_btn = gr.Button("Run research and draft email")

    email_markdown = gr.Markdown(label="Email (Markdown)")

    def _handle_run(topic, a1, a2, a3, send_flag, stored_questions):
        if not stored_questions:
            raise gr.Error("Generate clarifying questions first.")
        for update in on_run_pipeline(topic, [a1, a2, a3], stored_questions, send_flag):
            yield update

    run_btn.click(
        fn=_handle_run,
        inputs=[topic_input, answer1, answer2, answer3, send_toggle, questions_state],
        outputs=email_markdown,
    )


demo


In [ ]:
# Uncomment to launch the UI from the notebook
demo.launch(inbrowser=True)
